## Notebook 4.3: ReAct

In [ ]:
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

from src.client_tools import get_pod_log
from src import lab
from src.lab import get_llama_stack_client, run_session

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level1_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
client = get_llama_stack_client()

### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

We are going to switch LLM models here from the LLM llama32-3b model to the more powerful reasoning model deepseek-r1-0528-qwen3-8b-bnb-4bit that is able to more accurately reason about the tools it will use. Resoning models output their \<thought\> process first.

In [ ]:
agent = ReActAgent(
    client=client,
    model=lab.model_id,
    tools=[get_pod_log, "builtin::websearch"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=lab.sampling_params,
)

pod_name = 'java-app-build-iyyx5z-build-pod'
namespace = 'demo-application'
container_name = 'step-s2i-build'

user_prompts = [
    f"There's a pod {pod_name} in namespace {namespace} with a container {container_name}"
    f" that's been failing. Find out why it's been failing and provide a troubleshooting"
    f" recommendation based on public knowledge."
]
run_session(agent, 'react-session', user_prompts)

## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook: [Agents and MCP](5_mcp.ipynb).